In [7]:

# Some simple first steps
# - Aim is that notebook analysis can start with this simple code

from Malikowski.Adoption import Adoption

#-- specify the courses you want to analyse
#   Option to include (maybe)
#   - course = []  pass a list of courses
#   - PERIOD = []  pass a list of terms/periods e.g. [ 2015_1 2014_3 ]
#   - nothing - get all of them
# - ALTERNATIVE
#   Just support list of courses.  Use other course specific means to identify the specific
#   list of courses you're interested in for any analysis.

#courses = [6082]
#courses = []
courses = [ 6082, 4688, 4381]

#-- get the data
#   Eventually there will be different classess
#   - Adoption - standard adoption of features - from course_module table
#   - Usage - based on clicks of students
#   - ExtendedUsage - the extension to Malikowski where we put in extras
#          e.g. staff/student communication/content etc
# ...
model = Adoption()

model.getCourses( courses )

#-- Generates a range of data members
#   - df - (interim) data frame rows = count for 1 malikowski category for a course.
#   - malikowski - data frame with all course malikowski usage in one row
#model.df
model.malikowski

In [8]:

from Malikowski.AdoptionView import AdoptionView
plotly.offline.init_notebook_mode()

view = AdoptionView(model)
view.stackedBar()